In [10]:
!ln -s /Users/andrescrucettanieto/Documents/GitHub/missing_data_project/utils/utils.py utils.py

ln: utils.py: File exists


In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import nan_euclidean_distances
import utils

In [12]:
bikesy, bikesX = utils.read_in(utils.bike_file_path)
Xobs, Omega = utils.create_matrix(bikesX, keepcols=['instant', 'dteday'])

ValueError: could not convert string to float: '2011-01-01'